In [3]:
# General packages

import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import json
import re


In [ ]:
# Selenium

from selenium.webdriver.edge import service
from selenium import webdriver
from selenium.webdriver.common.by import By
from collections import Counter
from nltk.corpus import stopwords
from selenium.webdriver.common.action_chains import ActionChains

In [34]:
# Load scraped dataset
pages = pd.read_csv("data/full_pages.csv", ";", index_col = "Unnamed: 0")

/home/jane/miniconda2/envs/briq/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (19,20,21,22,23,24,25,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Collect penetration dates from the forum

## Collect mentions of product in other forums after the date

## Collect response rate to comments from forum

In [13]:
pages

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,open_issues,closed_issues,search_query
0,/t/app-6-0-compass-of-chance.4440211/,[APP][6.0+]Compass of Chance,"May 2, 2022","\nThe compass does not point to the north, but...",['https://play.google.com/store/apps/details?i...,Baiumka,/m/baiumka.12120441/,New member,"May 2, 2022",4,...,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN
1,/t/app-5-0-fairemail-fully-featured-open-sourc...,"[APP][5.0+] FairEmail - Fully featured, open s...","Aug 2, 2018","\n\n\n\n\n\nFairEmail\nOpen source, privacy fr...","['https://github.com/M66B/open-source-email/',...",M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,open-source-email
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,NaN,NaN,NaN,NaN,NaN,2.0,True,NaN,NaN,NaN
3,/t/xfinity-tv-mod-android-box-hdmi-output.3535...,Xfinity TV Mod Android Box (HDMI output),"Jan 11, 2017",\nIm modded an apk for Xfinity TV (new and lat...,[],aiemassfiria,/m/aiemassfiria.7985572/,Senior Member,"Jan 9, 2017",133,...,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NetGuard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38426,/t/market-as-separate-app.672499/,Market as separate app,"Apr 29, 2010","\nHey people, I rebooted my phone this morning...",[],fuz1987,/m/fuz1987.2336665/,Member,"Feb 3, 2010",20,...,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN
38427,/t/request-side-scrolling-online-game.672503/,(REQUEST) Side Scrolling online game,"Apr 29, 2010",\nIs it possible to create something along the...,[],fadentales,/m/fadentales.2219089/,Senior Member,"Dec 16, 2009",108,...,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN
38428,/t/1-50-bank-charge-on-every-app.672213/,£1.50 bank charge on every app.,"Apr 28, 2010","\nNot seen too many people moan about this, bu...",[],Phil750123,/m/phil750123.2481769/,Senior Member,"Apr 6, 2010",819,...,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN
38429,/t/request-relationship-helper-app.669411/,[REQUEST] Relationship helper app,"Apr 23, 2010",\nI need an app that will take care of the tex...,[],AdrianK,NaN,Senior Member,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,False,NaN,NaN,NaN


In [15]:
pages[pages["nonzero_price"]==True]

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_3,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,open_issues,closed_issues,search_query
1,/t/app-5-0-fairemail-fully-featured-open-sourc...,"[APP][5.0+] FairEmail - Fully featured, open s...","Aug 2, 2018","\n\n\n\n\n\nFairEmail\nOpen source, privacy fr...","['https://github.com/M66B/open-source-email/',...",M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,open-source-email
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,NaN,NaN,NaN,NaN,NaN,2.0,True,NaN,NaN,NaN
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NetGuard
5,/t/app-4-1-kiwi-browser-chromium-adblock-caf-n...,[APP][4.1+] Kiwi Browser? Chromium + Adblock +...,"May 30, 2018","\nHi,\n\n*** Join the Kiwi development Discord...","['https://discord.gg/XyMppQq', 'http://www.coi...",arnaud42,/m/arnaud42.9196003/,Senior Member,"May 30, 2018",446,...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN
28,/t/app-4-0-clone-app-multi-account-fake-gps-lo...,[APP] [4.0+]Clone App -Multi account&Fake GPS ...,"Jul 18, 2020",\nHey everyone! Thank you so much for all of ...,['https://mega.nz/file/slU22BgD#dZe1Dz2dM_j1Ea...,yule dee,/m/yule-dee.10713831/,Senior Member,"Apr 13, 2020",484,...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38330,/t/android-market-fixed-term-promotion.686535/,Android Market : Fixed-term promotion ?,"May 22, 2010","\nHi \nFirst, I'm sorry if I'm not on the righ...",[],bydox,/m/bydox.1029669/,Member,"Jun 13, 2008",16,...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN
38347,/t/wireless-itunes-syncing-for-android.686297/,Wireless Itunes Syncing for Android,"May 22, 2010","\nThis company offers an app called TuneSync, ...",[],jarod73,/m/jarod73.2567787/,Member,"May 11, 2010",42,...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN
38377,/t/the-washington-manual-of-medical-therapeuti...,The Washington Manual of Medical Therapeutics ...,"May 15, 2010",\nSkyscape recently released the 33rd edition ...,[],nathanc,/m/nathanc.2578517/,New member,"May 15, 2010",1,...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN
38404,/t/show-paid-apps-on-marketplace.676389/,show paid apps on marketplace ?,"May 5, 2010","\nhey, i live in Faroe Islands, and i can't se...",[],TheTmac,/m/thetmac.2026014/,Senior Member,"Sep 14, 2009",64,...,NaN,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN


In [35]:
pages.loc[pages.astype(str)['external_links'] != '[]', "ext_links_ind"] = True

In [28]:
pages["ext_links_ind"]

0        True
1        True
2        True
3         NaN
4        True
         ... 
38426     NaN
38427     NaN
38428     NaN
38429     NaN
38430     NaN
Name: ext_links_ind, Length: 38431, dtype: object

In [19]:
pages.loc[3, "url"]

'/t/xfinity-tv-mod-android-box-hdmi-output.3535881/'

In [36]:
pages.loc[pages["launch_message"].str.contains("http", regex = True), "ext_links_ind"] = True 

In [38]:
pages["ext_links_ind"].fillna(False)

0         True
1         True
2         True
3        False
4         True
         ...  
38426    False
38427    False
38428    False
38429    False
38430    False
Name: ext_links_ind, Length: 38431, dtype: bool

In [40]:
pages_ext = pages[pages["ext_links_ind"]==True]

In [42]:
pages_ext[pages_ext["nonzero_price"]==True]

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,open_issues,closed_issues,search_query,ext_links_ind
1,/t/app-5-0-fairemail-fully-featured-open-sourc...,"[APP][5.0+] FairEmail - Fully featured, open s...","Aug 2, 2018","\n\n\n\n\n\nFairEmail\nOpen source, privacy fr...","['https://github.com/M66B/open-source-email/',...",M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,open-source-email,True
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,NaN,NaN,NaN,NaN,2.0,True,NaN,NaN,NaN,True
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NetGuard,True
5,/t/app-4-1-kiwi-browser-chromium-adblock-caf-n...,[APP][4.1+] Kiwi Browser? Chromium + Adblock +...,"May 30, 2018","\nHi,\n\n*** Join the Kiwi development Discord...","['https://discord.gg/XyMppQq', 'http://www.coi...",arnaud42,/m/arnaud42.9196003/,Senior Member,"May 30, 2018",446,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
28,/t/app-4-0-clone-app-multi-account-fake-gps-lo...,[APP] [4.0+]Clone App -Multi account&Fake GPS ...,"Jul 18, 2020",\nHey everyone! Thank you so much for all of ...,['https://mega.nz/file/slU22BgD#dZe1Dz2dM_j1Ea...,yule dee,/m/yule-dee.10713831/,Senior Member,"Apr 13, 2020",484,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37895,/t/app-live-wallpaper-creator-free.748566/,[APP] - Live Wallpaper Creator - FREE -,"Aug 8, 2010","\nHello all, \n\nJust letting you all know abo...",['http://img844.imageshack.us/i/99367733.png/'...,potter1984,/m/potter1984.1482788/,Senior Member,"Jan 11, 2009",483,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
38153,/t/shapewriter-going-away.705992/,Shapewriter Going Away,"Jun 19, 2010",\nIt appears Shapewriter is leaving the Androi...,"['None', 'None', 'http://www.shapewriter.com/']",Vegasden,/m/vegasden.1984776/,Senior Member,"Aug 26, 2009","1,071",...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
38199,/t/general-mobile-application-market-storeoid....,General Mobile application market STOREOID,"Jun 24, 2010","\nHi Guys,\n\nGeneral Mobile released new Andr...","['http://www.storeoid.com', 'http://depositfil...",melgan,/m/melgan.1742628/,Member,"May 1, 2009",29,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
38249,/t/beta-app-tether-tetherberry-comes-to-androi...,[Beta App] Tether -- TetherBerry comes to And...,"Jun 15, 2010",\nI registered for the beta a few months back....,"['http://tether.com/a/setup.exe', 'http://teth...",magisterludi,/m/magisterludi.1725827/,Senior Member,"Apr 23, 2009",66,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True


In [46]:
pages_ext[(pages_ext["app"].isnull()==False)&(pages_ext["nonzero_price"]==True)]

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,open_issues,closed_issues,search_query,ext_links_ind
1,/t/app-5-0-fairemail-fully-featured-open-sourc...,"[APP][5.0+] FairEmail - Fully featured, open s...","Aug 2, 2018","\n\n\n\n\n\nFairEmail\nOpen source, privacy fr...","['https://github.com/M66B/open-source-email/',...",M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,open-source-email,True
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,NaN,NaN,NaN,NaN,2.0,True,NaN,NaN,NaN,True
4,/t/app-6-0-netguard-no-root-firewall.3233012/,[APP][6.0+] NetGuard - No-root firewall,"Oct 25, 2015",\n\n\n\n\n\n\nNetGuard provides simple and adv...,['https://contact.faircode.eu/?product=netguar...,M66B,/m/m66b.2799345/,Recognized Developer,"Aug 1, 2010","25,652",...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NetGuard,True
5,/t/app-4-1-kiwi-browser-chromium-adblock-caf-n...,[APP][4.1+] Kiwi Browser? Chromium + Adblock +...,"May 30, 2018","\nHi,\n\n*** Join the Kiwi development Discord...","['https://discord.gg/XyMppQq', 'http://www.coi...",arnaud42,/m/arnaud42.9196003/,Senior Member,"May 30, 2018",446,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
28,/t/app-4-0-clone-app-multi-account-fake-gps-lo...,[APP] [4.0+]Clone App -Multi account&Fake GPS ...,"Jul 18, 2020",\nHey everyone! Thank you so much for all of ...,['https://mega.nz/file/slU22BgD#dZe1Dz2dM_j1Ea...,yule dee,/m/yule-dee.10713831/,Senior Member,"Apr 13, 2020",484,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37607,/t/new-to-android-here-are-a-few-recommended-f...,New to Android? Here are a few RECOMMENDED FRE...,"Aug 1, 2010","\nFor those of you who are new to Android, I h...","['http://goo.gl/zTy7', 'http://twist.to/androi...",jahnile,/m/jahnile.2665379/,Senior Member,"Jun 17, 2010",273,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37630,/t/app-inventor-developers.777640/,App Inventor Developers,"Sep 8, 2010",\nUpon finding out that I can't upload my prog...,['http://www.xxthe3dmanxx.com'],the3dman,/m/the3dman.596909/,Senior Member,"Sep 5, 2007",657,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37895,/t/app-live-wallpaper-creator-free.748566/,[APP] - Live Wallpaper Creator - FREE -,"Aug 8, 2010","\nHello all, \n\nJust letting you all know abo...",['http://img844.imageshack.us/i/99367733.png/'...,potter1984,/m/potter1984.1482788/,Senior Member,"Jan 11, 2009",483,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
38199,/t/general-mobile-application-market-storeoid....,General Mobile application market STOREOID,"Jun 24, 2010","\nHi Guys,\n\nGeneral Mobile released new Andr...","['http://www.storeoid.com', 'http://depositfil...",melgan,/m/melgan.1742628/,Member,"May 1, 2009",29,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True


In [56]:
pages_ext[(pages_ext["app"].isnull()==False) & (pages_ext["launch_message"].str.contains("recommend|top|overview|digest")==False)&(pages["nonzero_price"] == True)]

/home/jane/miniconda2/envs/briq/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,open_issues,closed_issues,search_query,ext_links_ind
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,NaN,NaN,NaN,NaN,2.0,True,NaN,NaN,NaN,True
5,/t/app-4-1-kiwi-browser-chromium-adblock-caf-n...,[APP][4.1+] Kiwi Browser? Chromium + Adblock +...,"May 30, 2018","\nHi,\n\n*** Join the Kiwi development Discord...","['https://discord.gg/XyMppQq', 'http://www.coi...",arnaud42,/m/arnaud42.9196003/,Senior Member,"May 30, 2018",446,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
28,/t/app-4-0-clone-app-multi-account-fake-gps-lo...,[APP] [4.0+]Clone App -Multi account&Fake GPS ...,"Jul 18, 2020",\nHey everyone! Thank you so much for all of ...,['https://mega.nz/file/slU22BgD#dZe1Dz2dM_j1Ea...,yule dee,/m/yule-dee.10713831/,Senior Member,"Apr 13, 2020",484,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
62,/t/app-4-0-3-no-root-lan-drive-samba-fileshari...,[APP][4.0.3+][NO Root] LAN drive - Samba Files...,"May 15, 2018",\n\n\n\n\n\n\nIntroduction:\nLAN drive is an e...,['https://play.google.com/store/apps/details?i...,poppyto,/m/poppyto.1902945/,Senior Member,"Jul 17, 2009",58,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
65,/t/app-4-4-bromite-chromium-plus-adblocking.36...,[APP] [4.4+] Bromite - Chromium plus adblocking,"Oct 11, 2017",\nI have started an open source project to sup...,"['https://www.bromite.org/', 'https://forum.xd...",csagan5,/m/csagan5.8627832/,Senior Member,"Oct 11, 2017",442,...,NaN,NaN,NaN,NaN,1.0,True,26,"1,390",bromite,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37011,/t/true-caller-app.834055/,True Caller app,"Nov 9, 2010",\nThis app is pretty cool i just purchased it ...,['http://www.truecaller.com/?'],torvick12,/m/torvick12.2787827/,Senior Member,"Jul 28, 2010",488,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37307,/t/app-my-home-page.807780/,[APP] My Home Page,"Oct 13, 2010","\nI'd like to introduce my new app, My Home Pa...",['http://www.aaandroid.com/index.php/myhomepag...,BrooklynAvi,/m/brooklynavi.1325982/,Retired Recognized Developer,"Nov 5, 2008",498,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37392,/t/paid-apps-in-20-more-countries-devs-from-18...,Paid apps in 20 more countries & devs from 18 ...,"Oct 1, 2010",\nGOOD NEWS BROTHERS!!!!\nAndroid paid apps wi...,['http://tech.shantanugoel.com/2010/10/01/andr...,shantzg001,/m/shantzg001.307747/,Inactive Recognized Developer,"May 19, 2006",491,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37895,/t/app-live-wallpaper-creator-free.748566/,[APP] - Live Wallpaper Creator - FREE -,"Aug 8, 2010","\nHello all, \n\nJust letting you all know abo...",['http://img844.imageshack.us/i/99367733.png/'...,potter1984,/m/potter1984.1482788/,Senior Member,"Jan 11, 2009",483,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True


In [58]:
pages_nonzero_final = pages_ext[(pages_ext["app"].isnull()==False) & (pages_ext["launch_message"].str.contains("recommend|top|overview|digest|released")==False)&(pages["nonzero_price"] == True)]

/home/jane/miniconda2/envs/briq/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [59]:
pages_nonzero_final

,url,title,launch_date,launch_message,external_links,main_developer_username,main_developer_page,main_developer_title,main_developer_joined,main_developer_messages,...,contrlink_4,contrlink_5,contrlink_6,contrlink_7,number_of_developers,nonzero_price,open_issues,closed_issues,search_query,ext_links_ind
2,/t/app-all-root-solutions-6-0-viper4android-fx...,[APP][ALL ROOT SOLUTIONS][6.0+]ViPER4Android F...,"Apr 7, 2018",\nViPER4Android FX\n\n\n\n\n\nBy pittvandewitt...,['https://labs.xda-developers.com/store/app/co...,Team_DeWitt,/m/team_dewitt.9073985/,Senior Member,"Apr 7, 2018",71,...,NaN,NaN,NaN,NaN,2.0,True,NaN,NaN,NaN,True
5,/t/app-4-1-kiwi-browser-chromium-adblock-caf-n...,[APP][4.1+] Kiwi Browser? Chromium + Adblock +...,"May 30, 2018","\nHi,\n\n*** Join the Kiwi development Discord...","['https://discord.gg/XyMppQq', 'http://www.coi...",arnaud42,/m/arnaud42.9196003/,Senior Member,"May 30, 2018",446,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
28,/t/app-4-0-clone-app-multi-account-fake-gps-lo...,[APP] [4.0+]Clone App -Multi account&Fake GPS ...,"Jul 18, 2020",\nHey everyone! Thank you so much for all of ...,['https://mega.nz/file/slU22BgD#dZe1Dz2dM_j1Ea...,yule dee,/m/yule-dee.10713831/,Senior Member,"Apr 13, 2020",484,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
65,/t/app-4-4-bromite-chromium-plus-adblocking.36...,[APP] [4.4+] Bromite - Chromium plus adblocking,"Oct 11, 2017",\nI have started an open source project to sup...,"['https://www.bromite.org/', 'https://forum.xd...",csagan5,/m/csagan5.8627832/,Senior Member,"Oct 11, 2017",442,...,NaN,NaN,NaN,NaN,1.0,True,26,"1,390",bromite,True
67,/t/app-mod-pixel-launcher.4442077/,[APP][MOD] Pixel Launcher,"May 6, 2022",\n\n\n\n\nMod Pixel Launcher • Android 12 • An...,"['http://t.me/modulesrepo', 'https://telegra.p...",iamlooper,/m/iamlooper.11245295/,Senior Member,"Nov 14, 2020",62,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36820,/t/app-my-sms-related-apps.857182/,[APP] My SMS related-apps,"Dec 1, 2010",\nHello everyone.\nI would like to introduce s...,['http://www.net-vision.fr/android'],AlexKevler,/m/alexkevler.707690/,Member,"Nov 25, 2007",7,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37011,/t/true-caller-app.834055/,True Caller app,"Nov 9, 2010",\nThis app is pretty cool i just purchased it ...,['http://www.truecaller.com/?'],torvick12,/m/torvick12.2787827/,Senior Member,"Jul 28, 2010",488,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37307,/t/app-my-home-page.807780/,[APP] My Home Page,"Oct 13, 2010","\nI'd like to introduce my new app, My Home Pa...",['http://www.aaandroid.com/index.php/myhomepag...,BrooklynAvi,/m/brooklynavi.1325982/,Retired Recognized Developer,"Nov 5, 2008",498,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True
37392,/t/paid-apps-in-20-more-countries-devs-from-18...,Paid apps in 20 more countries & devs from 18 ...,"Oct 1, 2010",\nGOOD NEWS BROTHERS!!!!\nAndroid paid apps wi...,['http://tech.shantanugoel.com/2010/10/01/andr...,shantzg001,/m/shantzg001.307747/,Inactive Recognized Developer,"May 19, 2006",491,...,NaN,NaN,NaN,NaN,1.0,True,NaN,NaN,NaN,True


In [60]:
pages_nonzero_final.loc[2,"url"]

'/t/app-all-root-solutions-6-0-viper4android-fx-2-7.3774651/'